[MongoDB Lecture in Youtube](https://www.youtube.com/watch?v=s44QWegr2l8&list=PLWkguCWKqN9OwcbdYm4nUIXnA2IoXX0LI&index=2)

```
db.<collection>.aggregate([
    <stage1>,
    <stage2>,
    <stage3>,
    
    <stageN>
])
```

#### Server returns a cursor

In [ ]:
show collections
db.persons.aggregate([])

Each stage input documents, output documents <br>
Stages are independent

- Each stage starts from the stage operator
`{$<stageOperator> : {}}``
    - example
        - `{$match : {age : {$gt : 20}}}`
        - `{$group : {_id : "$age"}}`
        - `{$sort : {count : -1}}`

Expression refers to the name of the field in input document
- `"$<fieldname>"`

### Match stage

match uses standard MongoDB queries and supports all query operators

`{$match : {<query>}}`


In [ ]:
db.persons.aggregate([
    {$match : {age: {$gt:25}}}
])

In [ ]:
db.persons.aggregate([
    {$match : {isActive: true}}
])

하단 두 쿼리는 결과가 완전히 동일하다.

In [ ]:
db.persons.aggregate([
    {$match : {tags: {$size : 3}}}
])

db.getCollection('persons').find({tags: {$size : 3}})

### Group Stage

Groups input documents by certain expressions
- `{$group : {_id : <expression>, <field1> : {
    <accumulator1>: ,<expression1>}
    }`

- `
{$group : {_id : "$age"}}
` : Find all distinct of age values. produce new set of document. each document will have just one field

- `{$group : {_id : "$age", gender:"$gender"}}`

In [ ]:
db.persons.aggregate([
    {$group : {_id : "$age"}}
])

**group by nested field**

can use same as above. 

`db.persons.aggregate([
    {$group : {_id : "$company"}}
])
`

`db.persons.aggregate([
    {$group : {_id : "$company.location.country"}}
])
`

`db.persons.aggregate([
    {$group : {_id : "$company.location"}}
])
`

**group by several fields**
- all posible combinations

`db.persons.aggregate([
    { $group : { _id : {age:"$age", gender:"$gender"}}}
])`

`db.persons.aggregate([
    { $group : { _id : {eyeColor:"$eyeColor", favoriteFruit:"$favoriteFruit"}}}
])`

#### `$`match and `$`group

`db.persons.aggregate([
    //stage 1
    {$match : {favoriteFruit: "banana"}}, 
    // stage 2
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}}
])`

`db.persons.aggregate([
    {$match : {gender: "female"}}, 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}}
])`

`db.persons.aggregate([
    {$match : {gender: "female"}}, 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor", gender: "$gender"}}}
])`

아래는 stage순서를 바꾼 상태. 

`db.persons.aggregate([ 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}},
    { $match : {favoriteFruit: "banana"}}
])`

첫번째에서 group을 먼저 하면, 애초에 favoriteFruit라는 컬럼이 없다. fetch 0

아래처럼 하면 결과 나오겠지. 

`
db.persons.aggregate([ 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}},
    { $match : {"_id.age":{$gt:30}}}
])
`

`
db.persons.aggregate([ 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}},
    { $match : {"_id.eyeColor":"blue"}}
])
`

### Count

$count Stage

- counts number of the input documents

In [ ]:
{$count: "countries"}

In [ ]:
# 예약어

# Server-side Count

db.persons.aggregate([ 
    {$count : "allDocumentsCount"}
])

db.persons.aggregate([
    {$count: "total"}
])

db.persons.find({}).count() # Find Count() is wrapper of the Areggate $count

In [ ]:
# 아래 같은 자바스크립트 명령이 가능함.
# client-side count

db.persons.aggregate([]).toArray().length

In [ ]:
# client-side count

db.persons.aggregate([]).itcount()

db.persons.find({}).count()

In [ ]:
# countriesCount는 column명

db.persons.aggregate([
    {$group : {_id : "$company.location"}},
    {$count: "countriesCount"}
])

In [ ]:
db.persons.aggregate([
    {$group : {_id : "$eyeColor"}},
    {$count: "eyeColor"}
])

In [ ]:
db.persons.aggregate([
    {$group : {_id : {eyeColor:"$eyeColor", gender:"$gender"}}},
    {$count: "eyeColorAndgender"}
])

In [ ]:
db.persons.aggregate([
    {$match : { age : {$gte:25}}},
    {$group : {_id : {eyeColor:"$eyeColor", gender:"$gender"}}},
    {$count: "eyeColorAndgender"}
])


### $sort Stage

In [ ]:
# descending order
$sort : {score : -1} 
    
# age ascending order, and country ascending order
$sort : {age : 1, country: 1 } 

In [ ]:
db.persons.aggregate([
    {$sort : {name:1}}
])

In [ ]:
db.persons.aggregate([
    {$sort : {age:1}}
])

In [ ]:
db.persons.aggregate([
    {$sort : {age:-1}}
])

In [ ]:
db.persons.aggregate([
    {$sort : {age:-1, gender:-1, eyeColor:1}}
])

In [ ]:
db.persons.aggregate([
    {$group:{_id: "$favoriteFruit"}},
    {$sort : {_id:1}}
])

In [ ]:
db.persons.aggregate([
    {$group:{_id: "$age"}},
    {$sort : {_id:1}}
])

In [ ]:
# alphabetical order
db.persons.aggregate([
    {$group:{_id: "$eyeColor"}},
    {$sort : {_id:1}}
])

In [ ]:
db.persons.aggregate([
    {$group:{_id: {eyeColor:"$eyeColor", favoriteFruit: "$favoriteFruit"}}},
    {$sort : {"_id.eyeColor":1, "_id.favoriteFruit": -1}}
])

In [ ]:
db.persons.aggregate([
    {$match: {eyeColor: {$ne : "blue"}}},
    {$group:{_id: {eyeColor:"$eyeColor", favoriteFruit: "$favoriteFruit"}}},
    {$sort : {"_id.eyeColor":1, "_id.favoriteFruit": -1}}
])

### $project Stage

includes, exclues, or add new fields

In [ ]:
# _id, name, company.title
{$project : {name : 1, "company.title":1}}

# name, age
{$projject : {_id:0, name:1, age:1}}

# excludes, and others will be included
{$projject : {eyeColor:0, age:0}}

# _id, name, 그리고, age column will be renamed to newAge
{$project : {name:1, newAge: "$age"}}

In [ ]:
db.persons.aggregate([
    {$project : {name:1, "company.location.country":1}}
])

In [ ]:
db.persons.aggregate([
    {$project : {isActive:1, name:1, gender:1}}, 
    {$count: "total"}
])

In [ ]:
db.persons.aggregate([
    {$project : {_id:0, isActive:1, name:1, gender:1}}
])

아래처럼 하면 제외한 4개 빼고 다나옴. 

In [ ]:
db.persons.aggregate([
    {$project : {isActive:0, name:0, gender:0}}
])

rename도 가능. 구조 자체도 info 밑에 nested가 새로 생긴다. 

In [ ]:
db.persons.aggregate([
    { $project : {
        _id : 0, 
         name: 1,
        info:{
                eyes: "$eyeColor",
                fruit: "$favoriteFruit", 
                country: "$company.location.country"
            } 
                }}
])

### limit

<pre>
- sampled agregation requests with $limit as first state
- after $sort to produce topNresults

</pre>

In [ ]:
{$limit : 100}
{$limit : 1000 }

In [ ]:
# 체크하는 속도를 엄청 올릴 수 있다.
db.persons.aggregate([
    {$limit : 100}, 
    {$match: {age:{$gt:27}}},
    {$group : {_id:"$company.location.country"}}
])

In [ ]:
# 체크하는 속도를 엄청 올릴 수 있다.
# to use sample aggregation request
db.persons.aggregate([
    {$limit : 100},
    {$match : {eyeColor: {$ne: "blue"}}}, 
    {$group : {_id:{eyeColor:"eyeColor", favoriteFruit: "$favoriteFruit"}}},
    {$sort : {"_id.eyeColor":1, "_id.favoriteFruit": -1}}
])

In [ ]:
db.persons.aggregate([
    {$group : {_id : "$tags"}},

])

### unwind

이거는 상황 이해가 중요함. 아래 구조에서, 지금 tags하부에 있는 애들로 group by가 하고 싶은 상황.

![2_7.png](./resources/2_7.png)

그래서 아래 명령어를 입력하면

In [ ]:
db.persons.aggregate([
    {$group : {_id : "$tags"}},

])

![2_8.png](./resources/2_8.png)

내가 원한 결과가 아님. 

<pre>
{$unwind: <arrayReferenceExpression>}

{$unwind: "$tags"}

</pre>

![2_9.png](./resources/2_9.png)

document를 나눠준다. tags에 15개가 들어 있었으면, 15개로 나눠주는 것. 

In [ ]:
db.persons.aggregate([
    {$unwind: "$tags"}, 
    {$project: {name:1, gender:1, tags:1}}
])

![2_10.png](./resources/2_10.png)